In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

cuda


In [15]:
class Model(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,num_layers,batch_first=False):
        assert batch_first==True, "batch_first 값이 True일 때만 동작"
        
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=num_layers, batch_first=False)
        self.fc1 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.fc2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.fc3 = nn.Linear(hidden_size, output_size, bias=True)
        self.drop = nn.Dropout(0.25)
        self.activ = nn.ReLU()

    def forward(self, x):
        # Reshape Input Size for batch_first=True
        x = x.transpose(0, 1) # (B, L, I) -> (L, B, I)
    
        x, _status = self.rnn(x) # output x : (L, B, H)
        x = self.activ(self.drop(self.fc1(x))) # (L, B, H) -> (L, B, H)
        x = self.activ(self.drop(self.fc2(x))) # (L, B, H) -> (L, B, H)
        x = self.fc3(x) # (L, B, H) -> (L, B, O)
        
        # Reshape Output Size for batch_first=True
        x = x.transpose(0,1) # (L, B, O) -> (B, L, O)
        return x

In [16]:
epochs = 40
learning_rate = 0.01 # or use 1e-3

input_size = 1
hidden_size = 128
output_size = 1
seq_len = 24
num_layers = 4

In [17]:
model = Model(input_size, hidden_size, output_size, num_layers, batch_first=True).to(device)
print(model)

Model(
  (rnn): RNN(1, 128, num_layers=4)
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (drop1): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (drop2): Dropout(p=0.25, inplace=False)
  (fc3): Linear(in_features=128, out_features=1, bias=True)
)
